In [3]:
import torch
import torch.nn as nn
import pennylane as qml

In [4]:

#assert torch.cuda.is_available()
dev_torch = torch.device("cuda")

n_qubits = 4
n_layers = 2

qdev = qml.device("lightning.gpu", wires=n_qubits)

def circuit(inputs, weights):
    for i in range(n_qubits):
        qml.RX(inputs[i], wires=i)

    qml.StronglyEntanglingLayers(weights, wires=range(n_qubits))

    # >>> RETORNE UM ÚNICO TENSOR (4,) <<<
    obs = [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    return qml.math.stack(obs)  # <- chave

qnode = qml.QNode(
    circuit,
    qdev,
    interface="torch",
    diff_method="adjoint",
)

weight_shapes = {"weights": (n_layers, n_qubits, 3)}
quantum_layer = qml.qnn.TorchLayer(qnode, weight_shapes)

class HybridNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.q = quantum_layer
        self.fc = nn.Sequential(
            nn.Linear(n_qubits, 16),
            nn.Tanh(),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        q_feat = self.q(x)         # (batch, 4)
        return self.fc(q_feat)     # (batch, 1)

model = HybridNet().to(dev_torch)

# Dados
torch.manual_seed(0)
N = 512
X = (2 * torch.rand(N, n_qubits, device=dev_torch) - 1.0) * torch.pi
y = ((torch.sin(X[:, 0]) + torch.cos(X[:, 1])) > 0).float().unsqueeze(1)

opt = torch.optim.Adam(model.parameters(), lr=1e-2)
loss_fn = nn.BCEWithLogitsLoss()

for epoch in range(1, 51):
    opt.zero_grad(set_to_none=True)
    logits = model(X)
    loss = loss_fn(logits, y)
    loss.backward()
    opt.step()

    if epoch % 10 == 0:
        with torch.no_grad():
            acc = ((torch.sigmoid(logits) > 0.5) == y).float().mean().item()
        print(f"epoch={epoch:3d} loss={loss.item():.4f} acc={acc:.3f}")


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.